In [3]:
import json
import pickle
import numpy as np
import os
from typing import List, Dict, Any, Union

def load_polynomial_data(filename: str) -> List[Dict[str, Any]]:
    """
    다항식 데이터 파일을 로드합니다.
    
    Args:
        filename: 로드할 파일명 (.json 또는 .pkl)
        
    Returns:
        다항식 데이터 리스트
    """
    if not os.path.exists(filename):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {filename}")
    
    print(f"데이터 로드 중: {filename}")
    
    if filename.endswith('.json'):
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
    elif filename.endswith('.pkl'):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. (.json 또는 .pkl 사용)")
    
    print(f"로드 완료: {len(data)}개의 다항식 데이터")
    
    # 데이터 형태 확인 및 출력
    if data:
        sample = data[0]
        print(f"샘플 데이터 구조:")
        print(f"  계수: {len(sample['coefficients'])}개")
        print(f"  근 형태: {type(sample['roots'])}")
        
        # 근 데이터 구조 상세 분석
        roots = sample['roots']
        if isinstance(roots, list) and len(roots) > 0:
            if isinstance(roots[0], list):
                print(f"  근 구조: 2차원 리스트 [[실수부, 허수부], ...] - {len(roots)}개 근")
                print(f"  첫 번째 근 예시: {roots[0]}")
            else:
                print(f"  근 구조: 1차원 리스트 [실수부1, 허수부1, ...] - {len(roots)//2}개 근")
                print(f"  첫 번째 근 예시: [{roots[0]}, {roots[1]}]")
        
        print(f"  총 근 개수: {len(roots) if isinstance(roots[0], list) else len(roots)//2}")
    
    return data

def sort_roots_by_magnitude(roots: Union[List[List[float]], List[float]]) -> List[List[float]]:
    """
    복소수 근들을 크기(절댓값) 순으로 정렬합니다.
    
    Args:
        roots: [[실수부, 허수부], ...] 또는 [실수부1, 허수부1, 실수부2, 허수부2, ...] 형태
        
    Returns:
        크기 순으로 정렬된 근 리스트 [[실수부, 허수부], ...] 형태
    """
    # 데이터 형태 확인 및 변환
    if isinstance(roots[0], list):
        # 이미 [[실수부, 허수부], ...] 형태
        roots_array = np.array(roots)
    else:
        # [실수부1, 허수부1, ...] 형태를 [[실수부, 허수부], ...] 형태로 변환
        roots_array = np.array(roots).reshape(-1, 2)
    
    # 각 근의 크기(절댓값) 계산
    magnitudes = np.sqrt(roots_array[:, 0]**2 + roots_array[:, 1]**2)
    
    # 크기 순으로 정렬된 인덱스 구하기
    sorted_indices = np.argsort(magnitudes)
    
    # 정렬된 근 반환
    sorted_roots = roots_array[sorted_indices].tolist()
    
    return sorted_roots

def sort_roots_by_real_part(roots: Union[List[List[float]], List[float]]) -> List[List[float]]:
    """
    복소수 근들을 실수부 순으로 정렬합니다.
    
    Args:
        roots: [[실수부, 허수부], ...] 또는 [실수부1, 허수부1, 실수부2, 허수부2, ...] 형태
        
    Returns:
        실수부 순으로 정렬된 근 리스트 [[실수부, 허수부], ...] 형태
    """
    # 데이터 형태 확인 및 변환
    if isinstance(roots[0], list):
        roots_array = np.array(roots)
    else:
        roots_array = np.array(roots).reshape(-1, 2)
    
    # 실수부 기준으로 정렬
    sorted_indices = np.argsort(roots_array[:, 0])
    sorted_roots = roots_array[sorted_indices].tolist()
    
    return sorted_roots

def sort_roots_custom(roots: Union[List[List[float]], List[float]]) -> List[List[float]]:
    """
    사용자 정의 정렬: 실수근을 먼저, 그 다음 복소수근을 크기 순으로
    
    Args:
        roots: [[실수부, 허수부], ...] 또는 [실수부1, 허수부1, 실수부2, 허수부2, ...] 형태
        
    Returns:
        사용자 정의 순서로 정렬된 근 리스트 [[실수부, 허수부], ...] 형태
    """
    # 데이터 형태 확인 및 변환
    if isinstance(roots[0], list):
        roots_array = np.array(roots)
    else:
        roots_array = np.array(roots).reshape(-1, 2)
    
    # 실수근과 복소수근 분리
    real_roots = []
    complex_roots = []
    
    for i, root in enumerate(roots_array):
        if abs(root[1]) < 1e-10:  # 허수부가 거의 0인 경우 실수근
            real_roots.append((root, i))
        else:
            complex_roots.append((root, i))
    
    # 실수근은 실수부 기준으로 정렬
    real_roots.sort(key=lambda x: x[0][0])
    
    # 복소수근은 크기 기준으로 정렬
    complex_roots.sort(key=lambda x: np.sqrt(x[0][0]**2 + x[0][1]**2))
    
    # 결합
    sorted_roots = [root[0].tolist() for root in real_roots + complex_roots]
    
    return sorted_roots

def process_polynomial_data(data: List[Dict[str, Any]], 
                          sort_method: str = 'magnitude') -> List[Dict[str, Any]]:
    """
    다항식 데이터의 근들을 정렬합니다.
    
    Args:
        data: 원본 다항식 데이터
        sort_method: 정렬 방법 ('magnitude', 'real', 'custom')
        
    Returns:
        근이 정렬된 다항식 데이터
    """
    print(f"근 정렬 시작 ({sort_method} 방법)")
    
    processed_data = []
    
    for i, item in enumerate(data):
        coefficients = item['coefficients']
        roots = item['roots']
        
        # 정렬 방법에 따라 근 정렬
        if sort_method == 'magnitude':
            sorted_roots = sort_roots_by_magnitude(roots)
        elif sort_method == 'real':
            sorted_roots = sort_roots_by_real_part(roots)
        elif sort_method == 'custom':
            sorted_roots = sort_roots_custom(roots)
        else:
            raise ValueError("지원하지 않는 정렬 방법입니다. ('magnitude', 'real', 'custom' 중 선택)")
        
        # 새로운 데이터 항목 생성
        new_item = {
            'coefficients': coefficients,
            'roots': sorted_roots
        }
        
        # 기존에 다른 정보가 있다면 보존
        for key, value in item.items():
            if key not in ['coefficients', 'roots']:
                new_item[key] = value
        
        processed_data.append(new_item)
        
        # 진행 상황 출력
        if (i + 1) % 1000 == 0:
            print(f"처리 완료: {i + 1}/{len(data)}")
    
    print(f"근 정렬 완료: {len(processed_data)}개 데이터")
    return processed_data

def save_polynomial_data(data: List[Dict[str, Any]], 
                        filename: str, 
                        format_type: str = 'json') -> None:
    """
    정렬된 다항식 데이터를 저장합니다.
    
    Args:
        data: 저장할 데이터
        filename: 저장할 파일명
        format_type: 저장 형식 ('json' 또는 'pickle')
    """
    print(f"데이터 저장 중: {filename}")
    
    if format_type == 'json':
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
    elif format_type == 'pickle':
        with open(filename, 'wb') as f:
            pickle.dump(data, f)
    else:
        raise ValueError("지원하지 않는 저장 형식입니다. ('json' 또는 'pickle' 사용)")
    
    print(f"저장 완료: {filename}")

def analyze_roots_distribution(data: List[Dict[str, Any]]) -> None:
    """
    근들의 분포를 분석합니다.
    """
    print("\n=== 근 분포 분석 ===")
    
    all_magnitudes = []
    real_root_counts = []
    
    for item in data:
        roots = item['roots']
        
        # roots가 이미 리스트 형태인지 확인
        if isinstance(roots[0], list):
            # [[실수부, 허수부], ...] 형태
            roots_array = np.array(roots)
            magnitudes = np.sqrt(roots_array[:, 0]**2 + roots_array[:, 1]**2)
            all_magnitudes.extend(magnitudes)
            
            # 실수근 개수 세기
            real_count = sum(1 for root in roots if abs(root[1]) < 1e-10)
        else:
            # [실수부1, 허수부1, 실수부2, 허수부2, ...] 형태 (평탄화된 형태)
            roots_array = np.array(roots).reshape(-1, 2)
            magnitudes = np.sqrt(roots_array[:, 0]**2 + roots_array[:, 1]**2)
            all_magnitudes.extend(magnitudes)
            
            # 실수근 개수 세기
            real_count = sum(1 for i in range(0, len(roots), 2) if abs(roots[i+1]) < 1e-10)
        
        real_root_counts.append(real_count)
    
    all_magnitudes = np.array(all_magnitudes)
    real_root_counts = np.array(real_root_counts)
    
    print(f"근 크기 통계:")
    print(f"  평균: {np.mean(all_magnitudes):.4f}")
    print(f"  표준편차: {np.std(all_magnitudes):.4f}")
    print(f"  최소값: {np.min(all_magnitudes):.4f}")
    print(f"  최대값: {np.max(all_magnitudes):.4f}")
    
    print(f"\n실수근 개수 분포:")
    for i in range(6):  # 0~5개
        count = sum(1 for x in real_root_counts if x == i)
        percentage = count / len(real_root_counts) * 100
        print(f"  {i}개: {count}개 ({percentage:.1f}%)")

def compare_before_after(original_data: List[Dict[str, Any]], 
                        sorted_data: List[Dict[str, Any]], 
                        num_examples: int = 3) -> None:
    """
    정렬 전후 비교를 출력합니다.
    """
    print(f"\n=== 정렬 전후 비교 (첫 {num_examples}개 예시) ===")
    
    for i in range(min(num_examples, len(original_data))):
        print(f"\n--- 예시 {i+1} ---")
        print("계수:", original_data[i]['coefficients'])
        
        # 원본 근 데이터 처리
        original_roots = original_data[i]['roots']
        if isinstance(original_roots[0], list):
            # [[실수부, 허수부], ...] 형태
            orig_roots_array = original_roots
        else:
            # [실수부1, 허수부1, ...] 형태를 [[실수부, 허수부], ...] 형태로 변환
            orig_roots_array = [[original_roots[k], original_roots[k+1]] for k in range(0, len(original_roots), 2)]
        
        print("정렬 전 근들:")
        for j, root in enumerate(orig_roots_array):
            magnitude = np.sqrt(root[0]**2 + root[1]**2)
            if abs(root[1]) < 1e-10:
                print(f"  근 {j+1}: {root[0]:.6f} (크기: {magnitude:.6f})")
            else:
                print(f"  근 {j+1}: {root[0]:.6f} + {root[1]:.6f}i (크기: {magnitude:.6f})")
        
        # 정렬된 근 데이터 처리 (항상 [[실수부, 허수부], ...] 형태)
        print("정렬 후 근들:")
        for j, root in enumerate(sorted_data[i]['roots']):
            magnitude = np.sqrt(root[0]**2 + root[1]**2)
            if abs(root[1]) < 1e-10:
                print(f"  근 {j+1}: {root[0]:.6f} (크기: {magnitude:.6f})")
            else:
                print(f"  근 {j+1}: {root[0]:.6f} + {root[1]:.6f}i (크기: {magnitude:.6f})")

def main():
    """메인 함수"""
    # 설정
    input_file = "polynomial_dataset_2.json"  # 입력 파일명
    output_file = "polynomial_dataset_sorted.json"  # 출력 파일명
    sort_method = 'magnitude'  # 정렬 방법: 'magnitude', 'real', 'custom'
    
    try:
        # 1. 데이터 로드
        original_data = load_polynomial_data(input_file)
        
        # 2. 원본 데이터 분석
        analyze_roots_distribution(original_data)
        
        # 3. 근 정렬
        sorted_data = process_polynomial_data(original_data, sort_method)
        
        # 4. 정렬 전후 비교
        compare_before_after(original_data, sorted_data)
        
        # 5. 정렬된 데이터 저장
        save_polynomial_data(sorted_data, output_file, 'json')
        
        # 6. 추가로 pickle 형식으로도 저장
        pickle_file = output_file.replace('.json', '.pkl')
        save_polynomial_data(sorted_data, pickle_file, 'pickle')
        
        print(f"\n✅ 처리 완료!")
        print(f"   입력 파일: {input_file}")
        print(f"   출력 파일: {output_file}, {pickle_file}")
        print(f"   정렬 방법: {sort_method}")
        print(f"   처리된 데이터 수: {len(sorted_data)}")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

if __name__ == "__main__":
    # 사용자 입력으로 설정 변경 가능
    print("=== 다항식 근 정렬 도구 ===")
    print("1. magnitude: 크기(절댓값) 순 정렬")
    print("2. real: 실수부 순 정렬") 
    print("3. custom: 실수근 먼저, 복소수근은 크기 순")
    
    choice = input("\n정렬 방법을 선택하세요 (1/2/3, 기본값=1): ").strip()
    
    sort_methods = {'1': 'magnitude', '2': 'real', '3': 'custom'}
    sort_method = sort_methods.get(choice, 'magnitude')
    
    input_file = input("입력 파일명 (기본값=polynomial_dataset_2.json): ").strip()
    if not input_file:
        input_file = "polynomial_dataset_2.json"
    
    output_file = input("출력 파일명 (기본값=polynomial_dataset_sorted.json): ").strip()
    if not output_file:
        output_file = "polynomial_dataset_sorted.json"
    
    print(f"\n선택된 설정:")
    print(f"  정렬 방법: {sort_method}")
    print(f"  입력 파일: {input_file}")
    print(f"  출력 파일: {output_file}")
    
    confirm = input("\n계속 진행하시겠습니까? (y/N): ").strip().lower()
    if confirm == 'y':
        # 설정 업데이트
        globals()['input_file'] = input_file
        globals()['output_file'] = output_file
        globals()['sort_method'] = sort_method
        
        main()
    else:
        print("작업이 취소되었습니다.")

=== 다항식 근 정렬 도구 ===
1. magnitude: 크기(절댓값) 순 정렬
2. real: 실수부 순 정렬
3. custom: 실수근 먼저, 복소수근은 크기 순



정렬 방법을 선택하세요 (1/2/3, 기본값=1):  
입력 파일명 (기본값=polynomial_dataset_2.json):  
출력 파일명 (기본값=polynomial_dataset_sorted.json):  



선택된 설정:
  정렬 방법: magnitude
  입력 파일: polynomial_dataset_2.json
  출력 파일: polynomial_dataset_sorted.json



계속 진행하시겠습니까? (y/N):  y


데이터 로드 중: polynomial_dataset_2.json
로드 완료: 10000개의 다항식 데이터
샘플 데이터 구조:
  계수: 6개
  근 형태: <class 'list'>
  근 구조: 1차원 리스트 [실수부1, 허수부1, ...] - 5개 근
  첫 번째 근 예시: [0.0, 4.799145411552282]
  총 근 개수: 5

=== 근 분포 분석 ===
근 크기 통계:
  평균: 1.3591
  표준편차: 2.9466
  최소값: 0.0002
  최대값: 99.8154

실수근 개수 분포:
  0개: 10000개 (100.0%)
  1개: 0개 (0.0%)
  2개: 0개 (0.0%)
  3개: 0개 (0.0%)
  4개: 0개 (0.0%)
  5개: 0개 (0.0%)
근 정렬 시작 (magnitude 방법)
처리 완료: 1000/10000
처리 완료: 2000/10000
처리 완료: 3000/10000
처리 완료: 4000/10000
처리 완료: 5000/10000
처리 완료: 6000/10000
처리 완료: 7000/10000
처리 완료: 8000/10000
처리 완료: 9000/10000
처리 완료: 10000/10000
근 정렬 완료: 10000개 데이터

=== 정렬 전후 비교 (첫 3개 예시) ===

--- 예시 1 ---
계수: [18.96930664092995, -69.70450030956204, -95.37235879573667, -36.38915699474878, 31.7156084492905, -88.14267786403984]
정렬 전 근들:
  근 1: 0.000000 + 4.799145i (크기: 4.799145)
  근 2: -0.734883 + -1.010600i (크기: 1.249546)
  근 3: 0.734883 + -1.010600i (크기: 1.249546)
  근 4: -0.647389 + 0.448324i (크기: 0.787468)
  근 5: 0.647389 + 0.448324i (크기: 0.78